In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2 as cv
import torch
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchmetrics.functional import precision_recall
# from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
import pickle
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from sklearn.metrics import f1_score
import os
import random
import cv2 as cv

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
TRAIN_DIR = '../input/aptos2019-blindness-detection/train_images'
TRAIN_CSV = '../input/aptos2019-blindness-detection/train.csv'
train_df = pd.read_csv(TRAIN_CSV)
train_df.head()

In [ ]:
print(train_df.isna().sum())

In [ ]:
train_img1 = cv.imread('../input/aptos2019-blindness-detection/train_images/000c1434d8d7.png')
print(train_img1.shape)
plt.imshow(train_img1)

seen = []
for i in range(len(train_df)):
    img_fname = '../input/aptos2019-blindness-detection/train_images/' + train_df['id_code'][i] + '.png'
    img = cv.imread(img_fname)
    shape = img.shape
    if img.shape not in set(seen):
        seen.append(img.shape)
        
print(len(seen))

In [ ]:
class RetinopathyDataset(Dataset):
    def __init__(self, label_df, img_dir):
        self.imgs = [img_dir + '/' + fname + '.png' for fname in label_df['id_code']]
        self.labels = [label for label in label_df['diagnosis']]
        
    def __len__(self):
        return len(self.imgs)
        
    def __getitem__(self, i):
        img_arr = cv.imread(self.imgs[i])[:, 500:3000]
        
#         plt.imshow(img_arr)
        
        img_arr = resize(img_arr, (100, 100, 3)) * 255
        
        
        
        if np.max(img_arr) > 1:
            img_arr = img_arr/255.0

      # Reverse last three dimensions to get the shape: frame * rgb * h * w
        img_arr = np.moveaxis(img_arr, -1, 0)
#         print(img_arr.shape)

        
#         print(self.imgs[i], self.labels[i])
        return img_arr, self.labels[i]

In [ ]:
train_data = RetinopathyDataset(train_df[:(int(0.8 * len(train_df)))], TRAIN_DIR)
train_loader = DataLoader(dataset = train_data, batch_size = 16)

test_data = RetinopathyDataset(train_df[int(0.8 * len(train_df)):], TRAIN_DIR)
test_loader = DataLoader(dataset = test_data, batch_size = 16)

In [ ]:
import torchvision.models as models
import torch.nn as nn

resnet = models.resnet50(pretrained = True)

print(resnet)

In [ ]:
class IntensityDetector(nn.Module):
    def __init__(self):
        super(IntensityDetector, self).__init__()
        
        self.feature_extractor = models.resnet50(pretrained = True)
        for param in self.feature_extractor.parameters():
            param.requires_grad = False
            
        self.feature_extractor.fc = nn.Linear(in_features = 2048, out_features = 5, bias = True)  
        
    def forward(self, x):
        return self.feature_extractor(x)

In [ ]:
import torch

device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
model = IntensityDetector()
model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),
 lr=0.06, momentum=0.9)

In [ ]:
def calcMetrics(model, set_loader, file_name):
    y_pred = []
    y_true = []
    total = 0.0

  # visualize_filters()

  # accuracy = accuracy.cuda()
    with torch.no_grad():
        for data in set_loader:
            images, labels = data
              # run the model on the test set to predict labels
            outputs = model(images.float().cuda())
              # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputs.cuda().data, 1)

        for label, pred in list(zip(labels, predicted)):
            y_pred.append(pred.item())
            y_true.append(label.item())

  
    #   plot_confusion_matrix(y_true, y_pred, file_name)
    # precision_recall(torch.Tensor(y_pred).int().cuda(), torch.Tensor(y_true).int().cuda(), average='macro', num_classes=4)

    return f1_score(y_true, y_pred, average = 'macro')


In [ ]:
def train(model, epochs):
    best_acc = 0.0
    
    for epoch in range(epochs):
        running_loss = 0.0
        
        for i, (img, label) in enumerate(train_loader):
            img.to(device)
            label.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(img.float().cuda())
            
            loss = loss_fn(outputs.cuda(), label.cuda())
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            if i % 100:
                print('EPOCH %d loss: %f' %
                      (epoch + 1, running_loss / len(img)))
                running_loss = 0.0
            
#             file_name = f"/content/drive/MyDrive/cmatrix_epoch_test_{epoch}.png"
        test_acc = calcMetrics(model, test_loader, "test.png")
        f1_score = test_acc
        print('Test F1 score:', f1_score)
#         print('Precision:', pre_recall[0].item())
#         print('Recall:', pre_recall[1].item())
        print('-------------------------------------')
        print('-------------------------------------')
        
        if best_acc < f1_score:
            best_acc = f1_score

In [ ]:
train(model, 10)